前面改进的k均值聚类貌似效果不错,我就想用AP聚类来试一试,写这个算法废了不少的劲,不过效果不怎么好,因为我发现,一旦数据增多,这个算法分的类会急剧上升,当然数据量比较小的话,结果还是不错的.这里贴一下代码.

In [2]:
import numpy as np

class AffinityProp(object):
    """
        Implementing the affinity propagation algorithm
    """
    def __init__(self, similarity_matrix, max_iteration=200, num_iter=5,
    alpha=0.5, verbose=True, print_every=100):
        """
            similarity_matrix: N * N matrix containing similarities
            max_iteration: The maximum number of iterations to perfrom for clustering
            num_iter: num iterations without no change in the number of clusters
            that stops the algorithm
        """
        self.s = similarity_matrix # 得到相似矩阵
        self.max_iteration = max_iteration
        self.alpha = alpha
        self.verbose = verbose
        self.print_every = print_every

        #  INIITALISE THE RESPONSIBILITY AND THE AVAILABILITY MATRICES
        N, N = self.s.shape
        self.r = np.zeros((N, N)) # r, a两个矩阵在初始的时候全部是0
        self.a = np.zeros((N, N))

    def _step(self):
        """
            This is meant ot return the new Availability and repsonsibility
            matrices for all the data points
        """
        # 一次一次迭代
        N, N = self.s.shape # s是相似矩阵
        old_r = self.r
        old_a = self.a

        #R UPDATE STEP
        a_plus_s = self.a + self.s # 首先两个向量相加
        first_max = np.max(a_plus_s, axis=1) # 找到最大的
        first_max_indices = np.argmax(a_plus_s, axis=1)
        first_max = np.repeat(first_max, N).reshape(N, N)
        a_plus_s[range(N), first_max_indices] = -np.inf
        second_max =  np.max(a_plus_s, axis=1)
        r = self.s - first_max
        r[range(N), first_max_indices] = self.s[range(N), first_max_indices] - second_max[range(N)]
        r = self.alpha * old_r + (1 - self.alpha) * r

        # A UPDATE STEP
        rp = np.maximum(r, 0)
        np.fill_diagonal(rp, np.diag(r))
        a = np.repeat(np.sum(rp, axis=0), N).reshape(N,N).T - rp
        da = np.diag(a)
        a = np.minimum(a, 0)
        np.fill_diagonal(a, da)
        a = self.alpha * old_a + (1 - self.alpha) * a

        return r, a

    def solve(self):
        """
            This runs the affinity propagation algorithm until convergence
            TODO: make it into a separate file with more options for better update
            rules and more options
        """
        for i in range(self.max_iteration):
            if self.verbose and i % self.print_every is 0:
                print("processing iteration %d" % (i, ))
            self.r, self.a = self._step()

        e = self.r + self.a

        return e # 这个可以注释掉
        N, N = e.shape

        # NOTE: THIS IS ACCORDING TO THE PAPER
        # THIS IS NOT REMOTELY RELATED TO SCIKIT LEARN
        # SO I REALLY CANT COMPARE MY RESULTS TO SCIKIT LEARN BEYOND THIS POINT

        # I will contain the index of the data point that will be an exemplar
        # For example 40 and 55 of say 60 points may serve as the exemplars
        '''
        关于diag,给一个例子,可以明显看出diag用于取对角线的元素
        numpy.diag(v, k=0)
            Extract a diagonal or construct a diagonal array.
        Parameters
        v : array_like
        If v is a 2-D array, return a copy of its k-th diagonal. If v is a 1-D array, return a 2-D
        array with v on the k-th diagonal.
        k : int, optional
        Diagonal in question. The default is 0. Use k>0 for diagonals above the main diagonal,
        and k<0 for diagonals below the main diagonal.
        Returns
        out : ndarray
        The extracted diagonal or constructed diagonal array.
        >>> x = np.arange(9).reshape((3,3))
        >>> x
        array( [[0, 1, 2],
                [3, 4, 5],
                [6, 7, 8]])
        >>> np.diag(x)
        array([0, 4, 8])
        >>> np.diag(x, k=1)
        array([1, 5])
        >>> np.diag(x, k=-1)
        array([3, 7])
        >>> np.diag(np.diag(x))
        array( [[0, 0, 0],
                [0, 4, 0],
                [0, 0, 8]])
        '''
        # temp1 = np.diag(e)
        # temp2 = temp1 > 0
        # temp3 = np.where(temp2)
        # temp4 = temp3[0]
        I = np.where(np.diag(e) > 0)[0]
        K = len(I)

        # Select all the rows of S where column_index = 40 and 55
        c = self.s[:, I]
        # For every data point chose the exemplar that has maximum similarity with it
        # For example 1st data point may have maximum similarity with only 44
        # 2nd data point may have max similarity with 55
        # One explanation of why this may be done is to ensure that kth data
        # point not only maximises the sum a+r but also the similarity that it
        # has with i
        # so every c will be either 0 or 1 (considering only 2 exemplars are there)

        '''
        >>> a = np.arange(6).reshape(2,3)
        >>> a
        array( [[0, 1, 2],
                [3, 4, 5]])
        >>> np.argmax(a)
        5
        >>> np.argmax(a, axis=0)
        array([1, 1, 1])
        >>> np.argmax(a, axis=1)
        array([2, 2])
        >>> b = np.arange(6)
        >>> b[1] = 5
        >>> b
        array([0, 5, 2, 3, 4, 5])
        >>> np.argmax(b) # Only the first occurrence is returned.
        1
        '''
        c = np.argmax(c, axis=1)

        # Make the c[exemplar_1] = 0 and c[examplar_1] = 1
        c[I] = np.arange(0, K)

        # Get back the index to the original data set
        # say c= [0, 1, 1, 0]
        # 0th exemplar point is 40 in the original data and 1 is 55
        # mapping from c -> I is done like this in numpy
        idx = I[c]

        exemplar_indices = I
        exemplar_assignments = idx
        return exemplar_indices, exemplar_assignments

In [3]:
import numpy as np

'''
怎么说呢,这是一个很高级的矩阵计算,给定一个N*M维的矩阵,记作K,其中N代表元素的个数,M代表维数
我们要输出一个矩阵,这个矩阵N*N大小,记作G,例如G[1, 2]记录了G中第一个元素的第二个元素的欧几
里德距离,非常高效的算法
'''

def euclidean_distance(data, squared=True):
    """
        ARGS:
            INPUT:
                data: A matrix of shape (N, M)
                N -> Number of data points
                M -> Number of dimensions
            OUTPUT:
                similarity: of shape N * N representing the euclidean
                distance between the ith sample with all other samples
                Also here the sel similarity of the data point i is set to
                the median of the similarities with other data points
    """
    # N代表数据的个数,M代表维度
    N, M = data.shape
    # astype返回一个data的副本,但是元素被变成了float类型
    data = data.astype('float') # just making sure that it is float
    data_copy = data.copy() # 获得一个data的副本
    distance = np.zeros((N, N)) # distance是一个N*N的矩阵
    '''
    关于sum函数: numpy.sum(a, axis=None, dtype=None, out=None)
    a是array_like的元素
    axis是一个可选的参数,整数,默认的axis是None,从而a中所有的元素都被相加
    dtype也是一个可选的参数,决定结果的类型
    >>> np.sum([0.5, 1.5])
    2.0
    >>> np.sum([0.5, 0.7, 0.2, 1.5], dtype=np.int32)
    1
    >>> np.sum([[0, 1], [0, 5]])
    6
    >>> np.sum([[0, 1], [0, 5]], axis=0)
    array([0, 6])
    >>> np.sum([[0, 1], [0, 5]], axis=1)
    array([1, 5])
    '''
    data_square = np.sum(np.square(data), axis=1) # 得到向量的平方和
    # data_copy和data其实是一致的
    data_copy_square = np.sum(np.square(data_copy), axis=1)
    multiply = np.dot(data, data_copy.T) # 点乘,实际上是求两个矩阵相乘
    temp = data_square[:, np.newaxis]
    temp1 = 2 * multiply
    '''
    减法有一点诡异,举一个例子:
    >>> x1 = np.arange(9.0).reshape((3, 3))
    >>> x1
    array( [[ 0.,  1.,  2.],
            [ 3.,  4.,  5.],
            [ 6.,  7.,  8.]])
    >>> x2 = np.arrange(3.0)
    >>> x2
    array([ 0.,  1.,  2.])
    >>> np.subtract(x1, x2)
    array( [[ 0.,  0.,  0.],
            [ 3.,  3.,  3.],
            [ 6.,  6.,  6.]])
    '''
    temp3 = data_copy_square - temp1
    '''
    >>> np.add(1.0, 4.0)
    5.0
    >>> x1 = np.arange(9.0).reshape((3, 3))
    >>> x1
    array( [[ 0.,  1.,  2.],
            [ 3.,  4.,  5.],
            [ 6.,  7.,  8.]])
    >>> x2 = np.arange(3.0)
    >>> x2
    array([ 0.,  1.,  2.])
    >>> np.add(x1, x2)
    array( [[ 0., 2., 4.],
            [ 3., 5., 7.],
            [ 6., 8., 10.]])
    '''
    temp4 = temp + temp3
    distance = data_square[:, np.newaxis] + data_copy_square - (2 * multiply)

    if not squared:
        distance = np.sqrt(distance)

    return distance


if __name__ == "__main__":
    # Testing the euclidean similarity with simple matrices

    data = np.array([[1, 2, 3, 4], [5, 6, 7, 8]], dtype='float')
    N, M = data.shape
    correct_output = np.array([[0.0, 8.0], [8.0, 0.0]])
    similarity = euclidean_distance(data, squared=False)
    print("Are the two similarities equal %s" % (np.allclose(similarity, correct_output),))

    # Filling the diagonal of the euclidean distance
    # to the median but not including the self similarity points themselves
    # Using the masked array to find the median of the similarities
    masked_array = np.zeros((N, N), dtype='Bool')
    np.fill_diagonal(masked_array, True) # make the diagonals invalid
    ma_similarity = np.ma.masked_array(similarity, masked_array)
    np.fill_diagonal(similarity, np.ma.median(ma_similarity))
    correct_similarity_with_median = np.array([[8.0, 8.0], [8.0, 8.0]])
    print("Is similarities with medians equal %s" % (np.allclose(similarity, correct_similarity_with_median)))

Are the two similarities equal True
Is similarities with medians equal True


In [9]:
%matplotlib inline
import numpy as np

from sklearn.datasets.samples_generator import make_blobs
import matplotlib.pyplot as plt
import random

def plotClusters(clusters, title):
    """ 画图 """
    plt.figure(figsize=(8, 5), dpi=80)
    axes = plt.subplot(111)
    col=[]
    r = lambda: random.randint(0,255)
    for index in range(len(clusters)):
        col.append(('#%02X%02X%02X' % (r(),r(),r())))
    color = 0
    for cluster in clusters:
        cluster = np.array(cluster).T # T方法表示矩阵的转置
        axes.scatter(cluster[0],cluster[1], s=50, c = col[color])
        color += 1
    plt.title(title)

def computeCluster(fitable, data):
    clusters = {}
    num = len(fitable)
    for node in range(num):
        fit = list(fitable[node])
        key = fit.index(max(fit))
        if not key in clusters:
            clusters[key] = []
        point = tuple(data[node])
        clusters[key].append(point)
    return clusters

if __name__ == "__main__":

    n_clusters = 3
    centers = np.array([[1, 1], [-1, -1], [1, -1]]) + 10
    X, _ = make_blobs(n_samples=350, n_features=2, centers=centers,
                  cluster_std=0.4, shuffle=True, random_state=0)
    S = -1 * euclidean_distance(X, squared=True)
    median_value = np.median(S) * 10
    np.fill_diagonal(S, median_value)

    af_prop = AffinityProp(S)
    #exemplar_indices, exemplar_assignments = af_prop.solve()
    e = af_prop.solve()
    clusters = computeCluster(e, X)
    clusters = clusters.values()
    print(len(clusters))
    plotClusters(clusters, "clusters by affinity propagation")
    # print("cluster center indices mine: %s" % (exemplar_indices,))
    # print(len(exemplar_assignments))
    # print(np.unique(exemplar_assignments, return_counts=True))

    # RESULTS
    # THE INDICES OF THE EXEMPLARS IS 4 21 AND 22
    # THERE ARE 19, 20, 21 ELEMENTS IN EACH CLUSTER INCLUDING THE EXEMPLAR

在最后给出一张通过这个算法实现的非常漂亮的分类图片吧!一共大概350个点,但是如果点的数目再增加的话,分类效果就非常差了,所以我并不打算用这个聚类算法来分类我的数据.
![ap聚类](fig_4.png)